In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  GridSearchCV, train_test_split
from sklearn.metrics import log_loss
from featexp import get_univariate_plots, get_trend_stats

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
# from ngboost import NGBClassifier

%matplotlib inline

## Feature Engg

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
train.head(10)

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,class
0,0,1,150,36,144,172,947225,947332,439,439.099273,0,0,1,0,0,1
1,1,0,134,55,1144,1152,2379058,2379624,329,329.205616,1,0,0,0,0,1
2,1,0,135,65,950,974,1038442,1036754,300,300.120598,0,0,0,0,0,2
3,0,1,123,35,41,220,1705580,1705604,6803,6803.778622,0,0,1,0,0,1
4,1,0,138,69,462,466,1088124,1086579,251,251.401943,0,0,0,0,0,2
5,1,0,127,72,246,256,1137516,1136929,177,177.184821,0,1,0,0,0,1
6,0,1,99,39,242,359,87212,87169,3369,3369.037497,0,0,1,0,0,1
7,0,1,136,38,48,202,697615,696534,6159,6159.902918,0,0,1,0,0,1
8,0,1,121,43,42,214,4401789,4401986,5667,5667.675157,0,0,1,0,0,1
9,1,0,115,99,1289,1304,1005611,1006124,281,281.072866,1,0,0,0,0,1


In [4]:
train['specific_luminosity'] = train['max_luminosity']/(train['thickness']*train['log_area'])
train['delx*dely'] = (train['xmax']-train['xmin'])*(train['ymax']-train['ymin'])
train['pixel_luminosity'] = train['max_luminosity']/train['pixel_area']
train['pixel_pitch'] = train['pixel_area']**0.5
train['pixels'] = (train['xmax']*train['xmin'])/train['pixel_area']

In [5]:
test['specific_luminosity'] = test['max_luminosity']/(test['thickness']*test['log_area'])
test['delx*dely'] = (test['xmax']-test['xmin'])*(test['ymax']-test['ymin'])
test['pixel_luminosity'] = test['max_luminosity']/test['pixel_area']
test['pixel_pitch'] = test['pixel_area']**0.5
test['pixels'] = (test['xmax']*test['xmin'])/test['pixel_area']

In [6]:
train2 = train.iloc[:1000,:]
val2 = train.iloc[1000:,:]

print(train2.shape)
print(val2.shape)

(1000, 21)
(358, 21)


In [7]:
# get_univariate_plots(data=train2,target_col='class',data_test=val2,bins=10)

In [8]:
stats = get_trend_stats(data=train2,target_col='class',data_test=val2)

Returning stats for all numeric features


In [9]:
stats

,Feature,Trend_changes,Trend_changes_test,Trend_correlation
0,grade_A_Component_1,0,0,-1.000000
1,grade_A_Component_2,0,0,-1.000000
2,max_luminosity,3,5,0.750004
3,thickness,5,4,0.868085
4,xmin,5,5,0.855969
5,xmax,5,5,0.907552
6,ymin,5,4,0.812278
7,ymax,5,4,0.814904
8,pixel_area,3,4,0.737076
9,log_area,5,4,0.740626


In [10]:
train.columns

Index(['grade_A_Component_1', 'grade_A_Component_2', 'max_luminosity',
       'thickness', 'xmin', 'xmax', 'ymin', 'ymax', 'pixel_area', 'log_area',
       'x_component_1', 'x_component_2', 'x_component_3', 'x_component_4',
       'x_component_5', 'class', 'specific_luminosity', 'delx*dely',
       'pixel_luminosity', 'pixel_pitch', 'pixels'],
      dtype='object')

In [11]:
feature_list = [ 'max_luminosity',
       'thickness', 'xmin', 'xmax', 'ymin', 'ymax', 'pixel_area', 'log_area',
       'x_component_1', 'x_component_2', 'x_component_3', 'x_component_4',
       'x_component_5', 'specific_luminosity', 'delx*dely',
       'pixel_luminosity', 'pixel_pitch', 'pixels']

In [12]:
print(train.shape)
print(test.shape)

(1358, 21)
(583, 20)


In [13]:
X = train[feature_list]
y = train['class']
test2 = test[feature_list]

X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=101)

In [14]:
print(X.shape)
print(test2.shape)

(1358, 18)
(583, 18)


In [15]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_val = sc.fit_transform(X_val)
test3 = sc.fit_transform(test2)

In [16]:
def model_pred(model):
    model.fit(X_train,y_train)
    model_pred = model.predict_proba(X_val)
    model_score = log_loss(y_val,model_pred)
    return model_score, model_pred

In [17]:
def predis(model):
    model.fit(X,y)
    model_preds = model.predict_proba(test3)
    return model_preds

## Decision Trees

In [18]:
# dtc = DecisionTreeClassifier(min_samples_split=150,max_depth=20)

In [19]:
# dtc_score,dtc_val_pred = model_pred(dtc)

In [20]:
# dtc_score

In [21]:
# dtc_predis = predis(dtc)

## Random Forest

In [22]:
rfc = RandomForestClassifier(bootstrap=False,n_estimators=200,max_depth=20,min_samples_split=10,min_samples_leaf=2)

In [23]:
rfc_score,rfc_val_pred = model_pred(rfc)
rfc_score

0.3843117125089167

In [24]:
rfc_predis = predis(rfc)

## Extra Trees 

In [25]:
# etc = ExtraTreesClassifier(bootstrap=False,n_estimators=200,max_depth=20,min_samples_split=10,min_samples_leaf=2)

In [26]:
# etc_score,etc_val_pred = model_pred(etc)
# etc_score

In [27]:
# etc_predis = predis(etc)

## XGBoost

In [28]:
xgbc = XGBClassifier(learning_rate=0.1,n_estimators=100)

In [29]:
xgbc_score,xgbc_val_pred = model_pred(xgbc)
xgbc_score

0.33709399918306565

In [30]:
xgbc_predis = predis(xgbc)

ValueError: feature_names mismatch: ['max_luminosity', 'thickness', 'xmin', 'xmax', 'ymin', 'ymax', 'pixel_area', 'log_area', 'x_component_1', 'x_component_2', 'x_component_3', 'x_component_4', 'x_component_5', 'specific_luminosity', 'delx*dely', 'pixel_luminosity', 'pixel_pitch', 'pixels'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17']
expected specific_luminosity, xmin, xmax, pixel_area, x_component_3, delx*dely, ymax, max_luminosity, log_area, x_component_2, x_component_1, x_component_5, x_component_4, ymin, pixels, thickness, pixel_pitch, pixel_luminosity in input data
training data did not have the following fields: f0, f7, f6, f4, f1, f16, f17, f3, f9, f10, f8, f5, f2, f13, f14, f15, f11, f12

## LightGBM

In [31]:
lgbmc = LGBMClassifier(learning_rate=0.1,n_estimators=100)

In [32]:
lgbmc_score,lgbmc_val_pred = model_pred(lgbmc)
lgbmc_score

0.3991168702757925

In [33]:
lgbmc_predis = predis(lgbmc)

## CatBoost

In [34]:
cbc = CatBoostClassifier(learning_rate=0.1)

In [35]:
cbc_score,cbc_val_pred = model_pred(cbc)
cbc_score

0:	learn: 0.6247751	total: 86.8ms	remaining: 1m 26s
1:	learn: 0.5495799	total: 90.3ms	remaining: 45.1s
2:	learn: 0.5201216	total: 93.8ms	remaining: 31.2s
3:	learn: 0.4828518	total: 96.8ms	remaining: 24.1s
4:	learn: 0.4697328	total: 99.9ms	remaining: 19.9s
5:	learn: 0.4620638	total: 103ms	remaining: 17.1s
6:	learn: 0.4435327	total: 106ms	remaining: 15.1s
7:	learn: 0.4190924	total: 110ms	remaining: 13.6s
8:	learn: 0.4079409	total: 113ms	remaining: 12.5s
9:	learn: 0.3955004	total: 116ms	remaining: 11.5s
10:	learn: 0.3881502	total: 119ms	remaining: 10.7s
11:	learn: 0.3791890	total: 123ms	remaining: 10.1s
12:	learn: 0.3746407	total: 126ms	remaining: 9.54s
13:	learn: 0.3701196	total: 129ms	remaining: 9.06s
14:	learn: 0.3617483	total: 132ms	remaining: 8.7s
15:	learn: 0.3559903	total: 141ms	remaining: 8.67s
16:	learn: 0.3510715	total: 145ms	remaining: 8.36s
17:	learn: 0.3478170	total: 148ms	remaining: 8.05s
18:	learn: 0.3445420	total: 151ms	remaining: 7.8s
19:	learn: 0.3391306	total: 155ms	rem

176:	learn: 0.0965082	total: 907ms	remaining: 4.21s
177:	learn: 0.0958470	total: 917ms	remaining: 4.24s
178:	learn: 0.0952275	total: 927ms	remaining: 4.25s
179:	learn: 0.0945912	total: 933ms	remaining: 4.25s
180:	learn: 0.0938928	total: 938ms	remaining: 4.24s
181:	learn: 0.0930574	total: 947ms	remaining: 4.26s
182:	learn: 0.0926315	total: 966ms	remaining: 4.31s
183:	learn: 0.0920943	total: 972ms	remaining: 4.31s
184:	learn: 0.0914199	total: 980ms	remaining: 4.32s
185:	learn: 0.0903429	total: 985ms	remaining: 4.31s
186:	learn: 0.0895902	total: 990ms	remaining: 4.3s
187:	learn: 0.0891092	total: 996ms	remaining: 4.3s
188:	learn: 0.0879096	total: 1s	remaining: 4.31s
189:	learn: 0.0876297	total: 1.02s	remaining: 4.33s
190:	learn: 0.0868560	total: 1.02s	remaining: 4.33s
191:	learn: 0.0863236	total: 1.03s	remaining: 4.34s
192:	learn: 0.0858662	total: 1.03s	remaining: 4.32s
193:	learn: 0.0854699	total: 1.04s	remaining: 4.31s
194:	learn: 0.0848421	total: 1.04s	remaining: 4.3s
195:	learn: 0.0845

359:	learn: 0.0351748	total: 1.69s	remaining: 3.01s
360:	learn: 0.0350402	total: 1.7s	remaining: 3.01s
361:	learn: 0.0348844	total: 1.7s	remaining: 3s
362:	learn: 0.0347398	total: 1.71s	remaining: 3s
363:	learn: 0.0344264	total: 1.72s	remaining: 3s
364:	learn: 0.0342928	total: 1.72s	remaining: 3s
365:	learn: 0.0340894	total: 1.73s	remaining: 2.99s
366:	learn: 0.0338864	total: 1.73s	remaining: 2.98s
367:	learn: 0.0336806	total: 1.73s	remaining: 2.98s
368:	learn: 0.0334959	total: 1.74s	remaining: 2.97s
369:	learn: 0.0333353	total: 1.74s	remaining: 2.97s
370:	learn: 0.0330932	total: 1.75s	remaining: 2.96s
371:	learn: 0.0328772	total: 1.75s	remaining: 2.95s
372:	learn: 0.0328038	total: 1.75s	remaining: 2.95s
373:	learn: 0.0326744	total: 1.75s	remaining: 2.94s
374:	learn: 0.0325337	total: 1.76s	remaining: 2.93s
375:	learn: 0.0325053	total: 1.76s	remaining: 2.92s
376:	learn: 0.0323521	total: 1.77s	remaining: 2.92s
377:	learn: 0.0320989	total: 1.77s	remaining: 2.92s
378:	learn: 0.0319835	tota

566:	learn: 0.0165186	total: 2.48s	remaining: 1.89s
567:	learn: 0.0164476	total: 2.5s	remaining: 1.9s
568:	learn: 0.0164208	total: 2.5s	remaining: 1.9s
569:	learn: 0.0163824	total: 2.5s	remaining: 1.89s
570:	learn: 0.0163340	total: 2.51s	remaining: 1.89s
571:	learn: 0.0162948	total: 2.51s	remaining: 1.88s
572:	learn: 0.0162516	total: 2.52s	remaining: 1.87s
573:	learn: 0.0161913	total: 2.52s	remaining: 1.87s
574:	learn: 0.0161747	total: 2.52s	remaining: 1.86s
575:	learn: 0.0161133	total: 2.53s	remaining: 1.86s
576:	learn: 0.0161132	total: 2.53s	remaining: 1.86s
577:	learn: 0.0160907	total: 2.54s	remaining: 1.85s
578:	learn: 0.0160728	total: 2.54s	remaining: 1.84s
579:	learn: 0.0160444	total: 2.54s	remaining: 1.84s
580:	learn: 0.0160081	total: 2.54s	remaining: 1.83s
581:	learn: 0.0160013	total: 2.55s	remaining: 1.83s
582:	learn: 0.0159423	total: 2.55s	remaining: 1.82s
583:	learn: 0.0159190	total: 2.55s	remaining: 1.82s
584:	learn: 0.0159046	total: 2.56s	remaining: 1.81s
585:	learn: 0.015

726:	learn: 0.0111763	total: 3.28s	remaining: 1.23s
727:	learn: 0.0111589	total: 3.29s	remaining: 1.23s
728:	learn: 0.0111312	total: 3.29s	remaining: 1.22s
729:	learn: 0.0111073	total: 3.3s	remaining: 1.22s
730:	learn: 0.0110791	total: 3.3s	remaining: 1.22s
731:	learn: 0.0110605	total: 3.31s	remaining: 1.21s
732:	learn: 0.0110309	total: 3.31s	remaining: 1.21s
733:	learn: 0.0110036	total: 3.32s	remaining: 1.2s
734:	learn: 0.0109676	total: 3.32s	remaining: 1.2s
735:	learn: 0.0109523	total: 3.33s	remaining: 1.19s
736:	learn: 0.0109232	total: 3.33s	remaining: 1.19s
737:	learn: 0.0108970	total: 3.33s	remaining: 1.18s
738:	learn: 0.0108650	total: 3.34s	remaining: 1.18s
739:	learn: 0.0108333	total: 3.34s	remaining: 1.17s
740:	learn: 0.0108333	total: 3.35s	remaining: 1.17s
741:	learn: 0.0108034	total: 3.35s	remaining: 1.17s
742:	learn: 0.0107678	total: 3.35s	remaining: 1.16s
743:	learn: 0.0107164	total: 3.36s	remaining: 1.16s
744:	learn: 0.0106722	total: 3.36s	remaining: 1.15s
745:	learn: 0.01

909:	learn: 0.0080031	total: 4.71s	remaining: 466ms
910:	learn: 0.0079886	total: 4.72s	remaining: 461ms
911:	learn: 0.0079535	total: 4.73s	remaining: 457ms
912:	learn: 0.0079136	total: 4.74s	remaining: 452ms
913:	learn: 0.0078973	total: 4.75s	remaining: 447ms
914:	learn: 0.0078973	total: 4.76s	remaining: 442ms
915:	learn: 0.0078919	total: 4.77s	remaining: 437ms
916:	learn: 0.0078842	total: 4.78s	remaining: 432ms
917:	learn: 0.0078610	total: 4.78s	remaining: 427ms
918:	learn: 0.0078411	total: 4.79s	remaining: 422ms
919:	learn: 0.0078295	total: 4.8s	remaining: 417ms
920:	learn: 0.0078182	total: 4.81s	remaining: 413ms
921:	learn: 0.0077986	total: 4.82s	remaining: 408ms
922:	learn: 0.0077986	total: 4.82s	remaining: 402ms
923:	learn: 0.0077829	total: 4.83s	remaining: 397ms
924:	learn: 0.0077581	total: 4.84s	remaining: 393ms
925:	learn: 0.0077471	total: 4.87s	remaining: 389ms
926:	learn: 0.0077379	total: 4.88s	remaining: 385ms
927:	learn: 0.0077291	total: 4.9s	remaining: 380ms
928:	learn: 0.

0.4209207961891159

In [37]:
cbc_predis = predis(cbc)

0:	learn: 0.5948860	total: 4.08ms	remaining: 4.08s
1:	learn: 0.5240073	total: 7.91ms	remaining: 3.95s
2:	learn: 0.4927447	total: 11.6ms	remaining: 3.86s
3:	learn: 0.4687584	total: 15.3ms	remaining: 3.81s
4:	learn: 0.4545748	total: 18.7ms	remaining: 3.73s
5:	learn: 0.4394523	total: 22.2ms	remaining: 3.68s
6:	learn: 0.4251129	total: 25.7ms	remaining: 3.65s
7:	learn: 0.4082841	total: 29.1ms	remaining: 3.61s
8:	learn: 0.3994208	total: 32.7ms	remaining: 3.6s
9:	learn: 0.3911287	total: 36.4ms	remaining: 3.6s
10:	learn: 0.3815317	total: 40.4ms	remaining: 3.64s
11:	learn: 0.3721171	total: 43.9ms	remaining: 3.62s
12:	learn: 0.3664409	total: 48.4ms	remaining: 3.67s
13:	learn: 0.3619841	total: 52ms	remaining: 3.66s
14:	learn: 0.3546841	total: 55.8ms	remaining: 3.67s
15:	learn: 0.3483442	total: 59.2ms	remaining: 3.64s
16:	learn: 0.3448387	total: 62.8ms	remaining: 3.63s
17:	learn: 0.3423147	total: 66.4ms	remaining: 3.62s
18:	learn: 0.3401199	total: 69.8ms	remaining: 3.6s
19:	learn: 0.3350463	total:

194:	learn: 0.0943116	total: 786ms	remaining: 3.24s
195:	learn: 0.0938344	total: 792ms	remaining: 3.25s
196:	learn: 0.0932131	total: 797ms	remaining: 3.25s
197:	learn: 0.0927298	total: 802ms	remaining: 3.25s
198:	learn: 0.0921027	total: 814ms	remaining: 3.27s
199:	learn: 0.0917593	total: 819ms	remaining: 3.28s
200:	learn: 0.0912753	total: 823ms	remaining: 3.27s
201:	learn: 0.0908702	total: 828ms	remaining: 3.27s
202:	learn: 0.0899145	total: 832ms	remaining: 3.27s
203:	learn: 0.0891708	total: 836ms	remaining: 3.26s
204:	learn: 0.0889098	total: 839ms	remaining: 3.25s
205:	learn: 0.0888189	total: 843ms	remaining: 3.25s
206:	learn: 0.0882253	total: 847ms	remaining: 3.24s
207:	learn: 0.0880020	total: 850ms	remaining: 3.24s
208:	learn: 0.0872583	total: 854ms	remaining: 3.23s
209:	learn: 0.0869204	total: 857ms	remaining: 3.23s
210:	learn: 0.0861645	total: 863ms	remaining: 3.23s
211:	learn: 0.0855627	total: 867ms	remaining: 3.22s
212:	learn: 0.0851380	total: 870ms	remaining: 3.21s
213:	learn: 

359:	learn: 0.0409299	total: 1.57s	remaining: 2.8s
360:	learn: 0.0406464	total: 1.58s	remaining: 2.79s
361:	learn: 0.0404504	total: 1.58s	remaining: 2.79s
362:	learn: 0.0402160	total: 1.59s	remaining: 2.79s
363:	learn: 0.0399865	total: 1.59s	remaining: 2.78s
364:	learn: 0.0398118	total: 1.6s	remaining: 2.78s
365:	learn: 0.0396809	total: 1.6s	remaining: 2.78s
366:	learn: 0.0394831	total: 1.61s	remaining: 2.77s
367:	learn: 0.0392261	total: 1.61s	remaining: 2.77s
368:	learn: 0.0391015	total: 1.62s	remaining: 2.77s
369:	learn: 0.0389312	total: 1.62s	remaining: 2.76s
370:	learn: 0.0386406	total: 1.63s	remaining: 2.75s
371:	learn: 0.0384921	total: 1.63s	remaining: 2.75s
372:	learn: 0.0383551	total: 1.63s	remaining: 2.75s
373:	learn: 0.0382586	total: 1.64s	remaining: 2.74s
374:	learn: 0.0381448	total: 1.64s	remaining: 2.73s
375:	learn: 0.0378747	total: 1.65s	remaining: 2.73s
376:	learn: 0.0377488	total: 1.65s	remaining: 2.73s
377:	learn: 0.0376272	total: 1.65s	remaining: 2.72s
378:	learn: 0.0

552:	learn: 0.0202098	total: 2.36s	remaining: 1.91s
553:	learn: 0.0201646	total: 2.36s	remaining: 1.9s
554:	learn: 0.0201356	total: 2.37s	remaining: 1.9s
555:	learn: 0.0200858	total: 2.37s	remaining: 1.89s
556:	learn: 0.0200372	total: 2.38s	remaining: 1.89s
557:	learn: 0.0199606	total: 2.38s	remaining: 1.89s
558:	learn: 0.0199412	total: 2.38s	remaining: 1.88s
559:	learn: 0.0198951	total: 2.39s	remaining: 1.88s
560:	learn: 0.0198279	total: 2.39s	remaining: 1.87s
561:	learn: 0.0197667	total: 2.4s	remaining: 1.87s
562:	learn: 0.0197266	total: 2.4s	remaining: 1.86s
563:	learn: 0.0196813	total: 2.4s	remaining: 1.86s
564:	learn: 0.0196013	total: 2.41s	remaining: 1.85s
565:	learn: 0.0195320	total: 2.41s	remaining: 1.85s
566:	learn: 0.0194757	total: 2.42s	remaining: 1.84s
567:	learn: 0.0194419	total: 2.42s	remaining: 1.84s
568:	learn: 0.0193966	total: 2.42s	remaining: 1.83s
569:	learn: 0.0193278	total: 2.43s	remaining: 1.83s
570:	learn: 0.0193051	total: 2.43s	remaining: 1.82s
571:	learn: 0.019

734:	learn: 0.0125031	total: 3.16s	remaining: 1.14s
735:	learn: 0.0124823	total: 3.17s	remaining: 1.14s
736:	learn: 0.0124347	total: 3.17s	remaining: 1.13s
737:	learn: 0.0124139	total: 3.18s	remaining: 1.13s
738:	learn: 0.0123985	total: 3.19s	remaining: 1.13s
739:	learn: 0.0123881	total: 3.19s	remaining: 1.12s
740:	learn: 0.0123606	total: 3.2s	remaining: 1.12s
741:	learn: 0.0123415	total: 3.2s	remaining: 1.11s
742:	learn: 0.0123041	total: 3.2s	remaining: 1.11s
743:	learn: 0.0122778	total: 3.21s	remaining: 1.1s
744:	learn: 0.0122517	total: 3.21s	remaining: 1.1s
745:	learn: 0.0122090	total: 3.21s	remaining: 1.09s
746:	learn: 0.0121721	total: 3.22s	remaining: 1.09s
747:	learn: 0.0121578	total: 3.22s	remaining: 1.08s
748:	learn: 0.0121326	total: 3.23s	remaining: 1.08s
749:	learn: 0.0120974	total: 3.23s	remaining: 1.08s
750:	learn: 0.0120389	total: 3.23s	remaining: 1.07s
751:	learn: 0.0120087	total: 3.24s	remaining: 1.07s
752:	learn: 0.0119796	total: 3.24s	remaining: 1.06s
753:	learn: 0.011

905:	learn: 0.0089959	total: 4.14s	remaining: 430ms
906:	learn: 0.0089879	total: 4.15s	remaining: 425ms
907:	learn: 0.0089669	total: 4.15s	remaining: 421ms
908:	learn: 0.0089594	total: 4.16s	remaining: 416ms
909:	learn: 0.0089398	total: 4.16s	remaining: 412ms
910:	learn: 0.0088999	total: 4.17s	remaining: 407ms
911:	learn: 0.0088701	total: 4.17s	remaining: 402ms
912:	learn: 0.0088452	total: 4.17s	remaining: 398ms
913:	learn: 0.0088371	total: 4.18s	remaining: 394ms
914:	learn: 0.0088291	total: 4.19s	remaining: 389ms
915:	learn: 0.0088158	total: 4.19s	remaining: 384ms
916:	learn: 0.0087887	total: 4.19s	remaining: 380ms
917:	learn: 0.0087785	total: 4.2s	remaining: 375ms
918:	learn: 0.0087483	total: 4.2s	remaining: 370ms
919:	learn: 0.0087335	total: 4.2s	remaining: 366ms
920:	learn: 0.0087084	total: 4.21s	remaining: 361ms
921:	learn: 0.0086973	total: 4.21s	remaining: 356ms
922:	learn: 0.0086863	total: 4.22s	remaining: 352ms
923:	learn: 0.0086735	total: 4.22s	remaining: 347ms
924:	learn: 0.0

## Boosting predictions

In [38]:
boosted_predis = 0.2*rfc_predis+0.2*cbc_predis+0.6*lgbmc_predis

In [39]:
boost_predis = pd.DataFrame(boosted_predis,columns=['1','2'])

filename = 'submit_boosted_7.xlsx'
boost_predis.to_excel(filename,index=False)